# Week 3 Python Presentation on Data Manipulation

### Step 1: Install pandas library to create, read, and manipulate data files.

In [ ]:
pip install pandas

In [2]:
import pandas as pd

### Step 2: Create csv file

In [3]:
import csv
data = [
    ["Name", "Math Grade", "Science Grade", "English Grade"],
    ["Alice", 85, "Y4:0", 90],
    ["Bob", 78, "X2:0", 95],
    ["Charlie", 92, "O3:0", 88],
    ["David", 76, "N1:0", 92],
    ["Eve", 89, "P2:0", 91]
]

# Create the CSV file
with open('student_grades.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data)

print("CSV file 'student_grades.csv' has been created.")

CSV file 'student_grades.csv' has been created.


### Step 3: Read the csv file

In [4]:
try:
    df = pd.read_csv('student_grades.csv')
    print("File read successfully!")
except FileNotFoundError:
    print("Error: The file was not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File read successfully!


### Step 4: View data

In [5]:
print(df.head())
                 

      Name  Math Grade Science Grade  English Grade
0    Alice          85          Y4:0             90
1      Bob          78          X2:0             95
2  Charlie          92          O3:0             88
3    David          76          N1:0             92
4      Eve          89          P2:0             91


In [6]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           5 non-null      object
 1   Math Grade     5 non-null      int64 
 2   Science Grade  5 non-null      object
 3   English Grade  5 non-null      int64 
dtypes: int64(2), object(2)
memory usage: 292.0+ bytes
None


### Step 5: Initially Analyze Data/Identify Issues

In [7]:
math_avg = df["Math Grade"].mean()
print(f"Average Math Grade: {math_avg}")

Average Math Grade: 84.0


In [8]:
eng_avg = df["English Grade"].mean()
print(f"Average English Grade: {eng_avg}")

Average English Grade: 91.2


In [9]:
sci_avg = df["Science Grade"].mean()
print(f"Average Science Grade: {sci_avg}")

# error due to non-integer data type
# proceed to clean data

TypeError: Could not convert string 'Y4:0X2:0O3:0N1:0P2:0' to numeric

### Step 6: Clean data

The science grades were not displayed as integers, but instead in coded grades combining letters, numbers, and symbols. 
So, the codes were assigned values and a function was created to parse and interpret the coded grades.

Each code followed the following 4-part format:
LETTER NUMBER : 0

I wanted to create a function that split the coded grade at the colon and only retained the letter and number to the left of the colon, since the zero value would not add to the numerical grade.

For example:
Alice's science grade is 'Y4:0'
The function should convert 'Y4:0' to 'Y4'

Next, with assistance from generative AI, I assigned the letters numerical values, as follows:
        O (Outstanding) = 95-100
        X (Excellent) = 90-94
        Y (Good) = 85-89
        P (Pass) = 80-84
        N (Needs Improvement) = 75-79
I chose these values to not go below 75, based on patterns from existing  Math and English data that show scores being between 76 and 95.

With that, Alice's science grade looks like [85-89]4, well, actually [85]4.
The 4 will be used to decide where her grade falls within the given range.
So, the next step is for the function to add the adjacent value to the converted letter grade.
85 + 4 = 89.

Alice's science grade is 89.
Based on her Math grade of 85 and English grade of 90, this science grade figure passes the common sense test.


In [11]:
def clean_science_grade(grade):
    try:
        extracted_grade = grade.split(':') 
        if len(extracted_grade) != 2:
            return None
            
        # Get the letter and number (e.g., from 'Y4' get 'Y' and '4')
        grade_code = extracted_grade[0]
        if len(grade_code) < 2:
            return None
            
        letter = grade_code[0]
        # since the data came as a string, extracting the number still needs to be converted to an integer.
        # for scalability, we are extracting whatever is after the letter and before the colon split.
        # for example, if the coded grade was 'D10:0', this code would allow for the number to be 2 digits
        number = int(grade_code[1:])
        
        # assumed letter to number dictionary
        grade_scales = {
            'O': 95,
            'X': 90,
            'Y': 85,
            'P': 80,
            'N': 75
        }

        # error handling for letters that do not fall within above scale
        if letter not in grade_scales:
            return None
            
        # Calculate final grade
        # Base grade from letter + adjustment from number
        # Number ranges from 1-5, where 1 is highest
        base_grade = grade_scales[letter]
        number_adjustment = number  # To be added to base_grade
        
        final_grade = base_grade + number_adjustment
        
        return final_grade
        
    except (ValueError, TypeError, AttributeError) as e:
        print(f"Error processing grade {grade}: {str(e)}")
        return None

# Test the function
# test_grades = ['O3:0', 'X2:0', 'Y4:0', 'P2:0', 'N1:0']
# for grade in test_grades:
#     print(f"{grade} -> {clean_science_grade(grade)}")



In [12]:

# Apply the cleaning function to the Science Grade column
df['Science Grade'] = df['Science Grade'].apply(clean_science_grade)

# Save the updated DataFrame back to a CSV file
df.to_csv('student_grades_updated.csv', index=False)

print("Updated CSV file 'student_grades_updated.csv' has been created.")

Updated CSV file 'student_grades_updated.csv' has been created.


In [13]:
try:
    df_cleaned = pd.read_csv('student_grades_updated.csv')
    print("File read successfully!")
except FileNotFoundError:
    print("Error: The file was not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

File read successfully!


### Step 7: Calculate averages for Math, Science, and English using .mean()

In [14]:
avg_math_grade = df_cleaned["Math Grade"].mean()
print(f"The average math grade: {avg_math_grade}")

The average math grade: 84.0


In [15]:
avg_science_grade = df_cleaned["Science Grade"].mean()
print(f"The average science grade: {avg_science_grade}")

The average science grade: 87.4


In [16]:
avg_english_grade = df_cleaned["English Grade"].mean()
print(f"The average english grade: {avg_english_grade}")

The average english grade: 91.2


In [17]:
# to avoid issues with the 'Name' column, only select columns with integers for average calculation
numeric_columns = df_cleaned.select_dtypes(include='number')
subject_averages = numeric_columns.mean()

print(subject_averages)

Math Grade       84.0
Science Grade    87.4
English Grade    91.2
dtype: float64


In [18]:
highest_average_subject = subject_averages.idxmax()
highest_average_value = subject_averages.max()

print(f"Subject with the highest average: {highest_average_subject} ({highest_average_value})")

Subject with the highest average: English Grade (91.2)


### Step 6: Calculate averages by student across all subjects

In [19]:
# Calculate the average grade per student and add it as a new column
df_cleaned['average_student_grade'] = df_cleaned[["Math Grade", "Science Grade", "English Grade"]].mean(axis=1)

# Save the updated DataFrame with the new column to the same CSV file or a new file
df_cleaned.to_csv('student_grades_updated_averages.csv', index=False)

print(f"Updated CSV file with average_student_grade has been saved as 'student_grades_updated_averages.csv'.")

Updated CSV file with average_student_grade has been saved as 'student_grades_updated_averages.csv'.


### Step 7: Identify the overall highest achieving student using .max() --> .idxmax()

In [20]:
df_cleaned_averages = pd.read_csv('student_grades_updated_averages.csv')
print(df_cleaned_averages)

      Name  Math Grade  Science Grade  English Grade  average_student_grade
0    Alice          85             89             90              88.000000
1      Bob          78             92             95              88.333333
2  Charlie          92             98             88              92.666667
3    David          76             76             92              81.333333
4      Eve          89             82             91              87.333333


In [21]:
#idxmax(): Returns the index (row position) of the maximum value in a column, which allows you to access the corresponding row data (e.g., the student's name).
highest_achiever = df_cleaned_averages.loc[df_cleaned_averages['average_student_grade'].idxmax(), 'Name']
print(f"The highest achieving student is: {highest_achiever}!")


The highest achieving student is: Charlie!
